In [3]:
%load_ext jupyter_black

In [ ]:
import datetime
import subprocess

# Get hourly files from June 1 and June 2 2021
now = datetime.datetime(2021, 6, 1, 0)
utc_end = datetime.datetime(2021, 6, 2, 0)

while now < utc_end:
    cmd = (
        'wget -L -O ' + now.strftime('%Y%m%d%H') + '.zip ' +
        'https://mrms.agron.iastate.edu/' + now.strftime('%Y/%m/%d/%Y%m%d%H') + '.zip'
    )
    print(cmd)
    subprocess.run(cmd, shell=True)
    now += datetime.timedelta(hours=1)

In [10]:
import requests
import io
import zipfile
import shutil
from pathlib import Path
import uuid
import subprocess

url = "https://mrms.agron.iastate.edu/2022/08/23/2022082300.zip"
TMP_ROOT = Path("/tmp/mmmpy/")
TMP_DIR = "/tmp/mmmpy-tmp/"

class IAState:
    def __init__(self, targetdir=TMP_ROOT) -> None:
        self.targetdir = targetdir
        self.tmpdir = Path(TMP_DIR)

    def __enter__(self):

        if not self.tmpdir.exists():
            self.tmpdir.mkdir(parents=True)

        if not self.targetdir.exists():
            self.targetdir.mkdir(parents=True)

        return self

    def __exit__(self, *args):
        if self.tmpdir.exists():
            ...
            # shutil.rmtree(self.tmpdir)

    def get(self, url: str, raise_for_status=True):
        _, filename = url.rsplit("/", maxsplit=1)
        outfile = self.tmpdir / filename#.removesuffix(".zip")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with outfile.open("wb") as f:
                # shutil.copyfileobj(r.raw, fdst)
                for chunk in r.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
        # return self.__unzip(outfile)

    # def download_url(self, url: str, chunk_size=128):
    #     r = requests.get(url, stream=True)
    #     _, filename = url.rsplit("/", maxsplit=1)
    #     with (self.targetdir / filename).open("wb") as fd:
    #         for chunk in r.iter_content(chunk_size=chunk_size):
    #             fd.write(chunk)

    def __unzip(self, filename: str):
        """
        -FF
        --fixfix
            Fix the zip archive. The -F option can be used if some
            portions of the archive are missing, but requires a reasonably
            intact central directory.   The  input  archive is scanned as
            usual, but zip will ignore some problems.  The resulting
            archive should be valid, but any inconsistent entries will be
            left out.

            When doubled as in -FF, the archive is scanned from the
            beginning and zip scans  for  special  signatures  to
            identify  the  limits between the archive members. The single
            -F is more reliable if the archive is not too much damaged, so
            try this option first.

            If  the archive is too damaged or the end has been truncated,
            you must use -FF.  This is a change from zip 2.32, where the
            -F option is able to read a truncated archive.  The -F option
            now more reliably fixes archives with minor damage and the -FF
            option is  needed to fix archives where -F might have been
            sufficient before.
            ...
        """

        with subprocess.Popen(
            [
                "zip",
                "-FF",
                TMP_DIR + filename,
                # (self.tmpdir / filename).as_posix(),
                "--out",
                filename
            ],
        ):
            ...


# cmd = f"zip -FFv /tmp/mmmpy-de1beace-234e-11ed-b5a2-0242ac110002/2022082300.zip --out /tmp/mmmpy/2022082300.zip"
# subprocess.run(cmd, shell=True)
with IAState() as s:
    s.get(url)
# filename= "2022082300.zip"
# with subprocess.Popen(
#     [
#         "zip",
#         "-q",
#         "-FF",
#         TMP_DIR+filename,
#         # "/tmp/mmmpy-tmp/2022082300.zip",
#         "--out",
#         filename,
#     ]
# ):  ...

# assert False

In [8]:
tmp, = Path("/tmp/mmmpy-tmp/").glob("*")
with zipfile.ZipFile(tmp) as z:
    z.extractall("/tmp/mmmpy/")

BadZipFile: Bad magic number for file header

In [ ]:
import os

from zipfile import ZipFile


# def unzip (path, total_count):
#     for root, dirs, files in os.walk(path):
#         for file in files:
#             file_name = os.path.join(root, file)
#             if (not file_name.endswith('.zip')):
#                 total_count += 1
#             else:
#                 currentdir = file_name[:-4]
#                 if not os.path.exists(currentdir):
#                     os.makedirs(currentdir)
#                 with ZipFile(file_name) as zipObj:
#                     zipObj.extractall(currentdir)
#                 os.remove(file_name)
#                 total_count = unzip(currentdir, total_count)
#     return total_count
import gzip
f = Path("/tmp/mmmpy/2022082300.zip")

import zipfile
z = zipfile.ZipFile("/tmp/mmmpy/2022082300.zip")
# z.infolist()
# z.extractall()
f = [f.filename for f in z.infolist() if not f.filename.endswith(".gz")][0]
z.extract(member="2022082300/ANC/ANC_FinalForecast/")

# z.namelist()

In [ ]:
# f= open("/tmp/mmmpy/2022082300.zip")
zipfile.is_zipfile("/tmp/mmmpy/2022082300.zip")
# gzip.
with ZipFile("/tmp/mmmpy/2022082300.zip") as myzip:
    with myzip.open('eggs.txt') as myfile:
       eggs = io.TextIOWrapper(myfile)

In [ ]:
# r.content
import io
import zipfile

with io.BytesIO(r.content) as buffer:
    with zipfile.ZipFile(buffer) as z:
        z.extractall("/tmp/mmmpy")
# from pathlib import Path

# # import zipfile

# import shutil

# tmpdir = Path("/tmp/mmmpy")
# with (tmpdir / "2022082300.zip").open("wb") as fdst:
#     shutil.copyfileobj(r.raw, fdst)

In [ ]:
# tuple(tmpdir.glob("*"))file
import zipfile

with zipfile.ZipFile(tmpdir / "2022082300.zip", ) as z:
    print(z)

# file = tmpdir / "2022082300.zip"

In [ ]:
import re
import os
from pathlib import Path
import xarray as xr

FILE_NAME_PATTERN = re.compile(r"/([A-Za-z]+(?:-|_)?[A-Za-z]+)+")
data = Path(os.path.abspath(__name__)).parents[1] / "data"
data

In [ ]:
files = sorted((data / "MRMS_MergedReflectivity").glob("*grib2"))[:4]
files

In [ ]:
def dims(ds: xr.Dataset) -> xr.Dataset:
    duplicates = ["heightAboveSea"]
    # if more than one file was passed the valid_time should be greater than 1
    if ds.valid_time.size > 1:
        # for which we add a new validTime dimension
        ds = ds.expand_dims({"validTime": ds["valid_time"].to_numpy()})
        duplicates.append("validTime")

    return ds.drop("valid_time").drop_duplicates(duplicates)


def name(ds: xr.Dataset) -> xr.Dataset:
    if len(ds.data_vars) != 1:
        # mrms grib2 data should only have one variable
        raise Exception
    (ds_name,) = ds
    # not storing history, will use the history object to infer a name
    hist = ds.attrs.pop("history", None)
    # if a name was not explicility provided
    # if not name:
    # use the known name if unknow infer one from the file name
    if ds_name != "unknown":
        name = ds_name
    else:
        name_list = FILE_NAME_PATTERN.findall(hist)
        if name_list:
            name = name_list[-1]
        else:
            raise Exception

    return ds.rename({ds_name: name})


def main():
    ds = (
        xr.open_mfdataset(
            files,
            chunks={},
            engine="cfgrib",
            data_vars="minimal",
            combine="nested",
            concat_dim=["heightAboveSea"],
            backend_kwargs=dict(
                mask_and_scale=True,
                decode_times=True,
                concat_characters=True,
                decode_coords=True,
                # use_cftime="%Y-%m",
                decode_timedelta=None,
                lock=None,
                indexpath="{path}.{short_hash}.idx",
                filter_by_keys={},
                read_keys=[],
                encode_cf=("parameter", "time", "geography", "vertical"),
                squeeze=True,
                time_dims={"valid_time"},
            ),
        )
        .pipe(dims)
        .pipe(name)
    )
    return ds


# ds = main()
# ds

In [ ]:
mfds = ds.copy()
mfds.to_zarr(data / "mfds")

In [ ]:
xr.open_dataset(data / "mfds", engine="zarr")

In [ ]:
def generate():
    for file in files[:4]:
        ds1 = xr.open_dataset(
            file,
            engine="cfgrib",
            chunks={},
            # data_vars="minimal",
            # combine="nested",
            # concat_dim=["heightAboveSea"],
            backend_kwargs=dict(
                mask_and_scale=True,
                decode_times=True,
                concat_characters=True,
                decode_coords=True,
                # use_cftime="%Y-%m",
                decode_timedelta=None,
                lock=None,
                indexpath="{path}.{short_hash}.idx",
                filter_by_keys={},
                read_keys=[],
                encode_cf=("parameter", "time", "geography", "vertical"),
                squeeze=True,
                time_dims={"valid_time"},
            ),
        )
        yield ds1.expand_dims(
            {
                "validTime": [ds1["valid_time"].to_numpy()],
                "heightAboveSea": [ds1["heightAboveSea"].to_numpy()],
            }
        ).drop_duplicates(["validTime", "heightAboveSea"]).drop("valid_time").pipe(
            name
        )


datasets = tuple(generate())

In [ ]:
for x in xr.concat(datasets,dim="validTime").drop_duplicates("validTime").groupby("validTime"):
    print(x)

In [ ]:
import shutil
import zarr

teststore = data / "test-bucket"
group_name = "3DRefl"

if teststore.exists():
    shutil.rmtree(teststore)
import numpy as np
for vt, ds in xr.concat(datasets,dim="validTime").drop_duplicates("validTime").groupby("validTime"):
    (dsname,) = ds
    ds = ds.expand_dims({"validTime":[vt]}).fillna(np.nan)

    ds["MRMS_MergedReflectivityQC"].attrs.clear() 
    if not teststore.exists():
        ds.to_zarr(
            teststore,
            mode="a",
            group=group_name,
            compute=True,
            
        )
    else:
        ds.drop(["latitude","longitude","heightAboveSea"]).to_zarr(
            teststore,
            mode="a",
            group=group_name,
            append_dim="validTime",
            compute=True,
        )

xr.open_zarr(teststore / group_name, consolidated=False)

In [ ]:
import pandas as pd
paths = pd.Series((data / "MRMS_MergedReflectivity").glob("*.grib2"))
vt=pd.to_datetime(paths.apply(lambda p: p.name).str.extract(r"(\d{8}-\d{6})",expand=False))
for vt,x in paths.groupby(vt):
    print(tuple(x))
    # print(x)
# pd.to_datetime(for p in paths)#pd.Series(x.name for x in (data / "MRMS_MergedReflectivity").glob("*.grib2")).str.extract(r"(\d{8}-\d{6})",expand=False))

In [ ]:
# type(Group)
import zarr
root:zarr.Group = zarr.group(teststore / group)
def unpack_root()->tuple[zarr.Array,...]:
    yield from (root[key] for key in ("latitude","longitude","heightAboveSea","validTime"))
x,y,z,t = unpack_root()
tuple(z)
# x,y,z,t :tuple[zarr.Array,...]= root["latitude"], root["longitude"],  root["heightAboveSea"], root["validTime"],
# x,y
# z.values
# g:zarr.Group = zarr.open(teststore / group)
# tuple(g.array_keys())
# g.info.obj
# g.info_items()
# g.store.items()
# g.c
# vt: np.datetime64 = ds["validTime"].values[0]
# (vt,) = pd.to_datetime(ds["validTime"].values).strftime("%Y-%m-%dT")

# ds["validTime"].to_numpy().astype("datetime64[m]")
# vt
# np.str
# import zarr

# g: zarr.Group = zarr.open(teststore / "3DRefl")
# cs: zarr.DirectoryStore = g.chunk_store
# g.get("validTime").size
# # print(g.get("validTime"))
# # tuple(g)